<a href="https://colab.research.google.com/github/yosunokoji/HP/blob/main/NDL%E5%8F%A4%E5%85%B8%E7%B1%8DOCR_v2%E3%81%AE%E5%AE%9F%E8%A1%8C%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colaboratoryのアップデートなどにより、本ノートブックが実行できない場合があります。その場合、以下の稼働状況をご確認ください。

https://ldas.statuspage.io/

<br/>

また成果の共有にご協力いただけますと幸いです。 <a href="https://mahalo.ex.nii.ac.jp/button/0136d878-f3a0-4144-af0a-fa2fa7c309ff/mahalo">Mahalo Button</a> by DIAS (Data Integration and Analysis).

<br/><br/>

# <font color="green">NDL古典籍OCR（ver.3）の実行例</font>

本ノートブックのライセンス： <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />

<br/>

NDL古典籍OCRの説明: https://github.com/ndl-lab/ndlkotenocr_cli

<br/>

本ノートブックでは、Google Driveに認識結果を出力します。本ノートブックの説明については、以下を参考にしてください。

https://zenn.dev/nakamura196/articles/43151b473e8954

<br/>

## 参考にしたノートブック

@blue0620 さんが作成したノートブック

https://github.com/blue0620/NDLkotenOCR-GoogleColabVersion/blob/main/NDLkotensekiOCR_googlecolabversion.ipynb

<br/>

## 更新内容
- 2024-08-21
  - NDL古典籍OCRアプリケーションのバージョンを2から3に変更しました。
- 2023-10-23
 - ステータスページへのリンクを追加しました。
 - エラーのハンドリングを修正しました。
 - 不具合を修正しました。
- 2023-10-17
 - IIIFマニフェストのコマ数を指定する際の不具合を修正しました。
- 2023-09-24
 - PDF出力時の不具合を修正しました。
- 2023-09-23
 - 横書きテキストに対するPDF出力の不具合を修正しました。
- 2023-09-19
 - ノートブックを公開しました。
- 2023-08-25
 - Miradorで10ページ以降の画像が表示されない不具合を修正しました。

<br/><br/><br/><br/>

## 1.初期セットアップ

少し時間がかかります。初回のみ実行が必要です。

In [ ]:
#@title セットアップ1
from pathlib import Path
CONTENT_DIR = str(Path("/content"))

from IPython.display import clear_output

# Google Drive関連
from google.colab import drive
drive.mount('/content/drive/')

#

!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 torchtext==0.16.1 --index-url https://download.pytorch.org/whl/cu121

# Googleドライブのパスを取得
!pip install kora
from kora.xattr import get_id

def message(path):
  print("以下に出力しました。\nhttps://drive.google.com/drive/folders/{}".format(get_id(path)))

%cd {CONTENT_DIR}

# OCR関連のセットアップ
!git clone --recursive https://github.com/ndl-lab/ndlkotenocr_cli -b feature/colab
PROJECT_DIR="/content/ndlkotenocr_cli"

!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html


!pip install mmdet==3.3.0
!pip install mmpretrain==1.2.0
!pip install transformers
!pip install torchmetrics==0.11.4

%cd {PROJECT_DIR}
!wget -nc https://lab.ndl.go.jp/dataset/ndlkotensekiocr/trocr/model-ver2.zip -P ./src/text_kotenseki_recognition/
!wget -nc https://lab.ndl.go.jp/dataset/ndlkotensekiocr/layoutmodel/ndl_kotenseki_layout_ver3.pth -P ./src/ndl_kotenseki_layout/models/
!unzip -o ./src/text_kotenseki_recognition/model-ver2.zip -d ./src/text_kotenseki_recognition/

# %cd /content/

clear_output()

DOCS_DIR = f"{CONTENT_DIR}/docs"

from google.colab import output

PORT = 8001
!mkdir -p {DOCS_DIR}
%cd {DOCS_DIR}
!nohup python3 -m http.server $PORT > server.log 2>&1 &

if not Path("index.html").exists():
  !wget https://nakamura196.github.io/mirador-integration-textoverlay/index.html -O index.html
  !mkdir -p dist
  !wget https://nakamura196.github.io/mirador-integration-textoverlay/dist/main.js -O dist/main.js

!apt-get install poppler-utils
!pip install pdf2image
!pip install ldas==0.0.8
!pip install ocr_iiif_tools==0.0.16
!pip install scikit-learn==1.2.2

clear_output()

In [ ]:
#@title セットアップ2

import uuid
import yaml
import datetime
import requests
import os
from pathlib import Path
from pdf2image import convert_from_path
import os
from google.colab import files
import glob
import sys
from ldas.iiif import IIIF
from ocr_iiif_tools.pdf import PdfClient
import subprocess
import json
from bs4 import BeautifulSoup
from PIL import Image

# google colab依存
def upload_get_input_file():
  uploaded = files.upload()
  input_file = next(iter(uploaded))
  return input_file

class Task:
  docs_dir = "/content/docs"

  project_dir = "/content/ndlkotenocr_cli"

  def __init__(self, output_dir, task_id = None, docs_dir = None, project_dir = None):
    # self.convert_process_to_range(process)

    if docs_dir != None:
      self.docs_dir = docs_dir

    if project_dir != None:
      self.project_dir = project_dir

    self.files_dir = f"{self.docs_dir}/files"

    if task_id is None:
      self.generate_process_id()
    else:
      self.task_id = task_id

    self.get_temporary_directory()
    self.prepare_img_directory()
    self.prepare_output_directory(output_dir)

  def generate_process_id(self):
    self.task_id = uuid.uuid1()

  def get_temporary_directory(self):
    self.temporary_directory = f"{self.files_dir}/{self.task_id}"

  def prepare_img_directory(self):
    target_dir_path = Path(self.temporary_directory)
    img_dir_path = target_dir_path / "img"
    img_dir_path.mkdir(parents=True, exist_ok=True)
    self.img_dir_path = str(img_dir_path)

  def prepare_output_directory(self, output_dir):
    output_dir_path = Path(output_dir)

    if output_dir_path.exists():
      dt_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
      dt_str = dt_now.isoformat()
      new_folder_name = str(output_dir_path) + "_" + dt_str
      output_dir_path = Path(new_folder_name)

    output_dir_path.parent.mkdir(parents=True, exist_ok=True)
    self.output_dir_path = str(output_dir_path)

  def download_image(self, url, filename):
    output_path = self.img_dir_path + "/" + filename
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            file.write(response.content)

  @staticmethod
  def convert_json_to_xml(task_id, img_dir, output_dir):

    output_task_dir = f"{output_dir}/{task_id}"

    json_dir = f"{output_task_dir}/json"

    files = glob.glob(img_dir + "/*.jpg")

    files.sort()

    soup = BeautifulSoup("", 'xml')
    ocr = soup.new_tag("OCRDATASET")
    soup.append(ocr)

    for file in files:
      filename = file.split("/")[-1].split(".")[0]

      im = Image.open(file)
      w, h = im.size

      json_path = json_dir + "/" + filename + ".json"
      with open(json_path, "r") as f:
        df = json.load(f)

      # print(df)


      page = soup.new_tag("PAGE",attrs={
          "IMAGENAME": file.split("/")[-1],
          "WIDTH" : w,
          "HEIGHT" : h
          })
      ocr.append(page)

      block = soup.new_tag("TEXTBLOCK ")
      page.append(block)

      bbs = df[0]

      for i in range(len(bbs)):
        e = bbs[i]

        line = soup.new_tag("LINE",attrs={
          "TYPE": "本文",
          "X" : e[0],
          "Y" : e[1],
          "WIDTH": e[2] - e[0],
          "HEIGHT": e[3] - e[1],
          "STRING": e[4],
          "ORDER": i
          })

        block.append(line)

    xml_path = f"{output_task_dir}/xml/{task_id}.xml"

    os.makedirs(os.path.dirname(xml_path), exist_ok=True)
    f = open(xml_path, "w")
    f.write(str(soup))
    f.close()


  @staticmethod
  def show_mirador(output_path):
    print("\n認識結果は以下です。")
    path = f'/?manifest={output_path.replace("/content/docs", "")}&annotationState=1' # .replace("/content/docs", "")
    # output.serve_kernel_port_as_window(PORT, path=path) # , path=f'/?manifest={output_path.replace("/content", "")}&annotationState=1'
    output.serve_kernel_port_as_iframe(PORT, path=path, width="100%", height="600px")

  def run_common_pipeline(self, services=None):
    %cd {self.project_dir}

    cmd = f'python main.py infer "{self.temporary_directory}" "{self.output_dir_path}" -s s'
    result = subprocess.run(cmd, cwd=self.project_dir, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        raise Exception(f"Error: {result.stderr}")

    Task.convert_json_to_xml(self.task_id, self.img_dir_path, self.output_dir_path)

    ###

    task = self

    root_path =task.docs_dir
    output_dir = str(task.output_dir_path)
    task_id = str(task.task_id)

    def copy_xml(output_dir, task_id):

      xml_path = f"{output_dir}/{task_id}/xml/*.xml"
      xml_files = glob.glob(xml_path)

      for xml_file in xml_files:
        if ".sorted" in xml_file:
          !cp {xml_file} {xml_file.replace(".sorted", "")}

    copy_xml(output_dir, task_id)

    IIIF.ndl_ocr_post_process_simple(root_path, output_dir, task_id, services=services)

    ###

    client = PdfClient()
    output = f"{output_dir}/{task_id}/pdf/{task_id}.pdf"
    input_path = f"{root_path}/iiif/{task_id}/manifest_a.json"
    client.convert_iiif2pdf(output, iiif_manifest_path=input_path, image_download_dir=f"{root_path}/files/{task_id}/img")

    output_text = f"{output_dir}/{task_id}/pdf/{task_id}_text.pdf"
    client.convert_iiif2pdf(output_text, iiif_manifest_path=input_path, image_download_dir=f"{root_path}/files/{task_id}/img", default_alpha=0.4)

    ###

    clear_output()

    ###

    message(f"{output_dir}/{task_id}")

    ###

    output_manifest_alto_path = f"{root_path}/iiif/{task_id}/manifest_o.json"
    Task.show_mirador(output_manifest_alto_path)

    ### コピー

    iiif_in = f"{root_path}/iiif/{task_id}"
    iiif_out = f"{output_dir}/{task_id}/iiif"

    !cp -rp {iiif_in} {iiif_out}

    # 画像の削除
    !rm -rf {output_dir}/{task_id}/pred_img
    !rm -rf {output_dir}/{task_id}/img

  def process_image_from_url(self, url):

    self.run_common_pipeline()

  def prepare_pdf(self, pdf_path_):
    # pdfのパス
    pdf_path = Path(pdf_path_)

    # 変換
    img_dir = self.img_dir_path
    os.makedirs(img_dir, exist_ok=True)

    convert_from_path(pdf_path, output_folder=img_dir, fmt='jpeg', output_file=pdf_path.stem)

    # 画像のリネーム
    for idx, img_file in enumerate(sorted(Path(img_dir).glob(f"{pdf_path.stem}*.jpg")), 1):
        target_name = Path(img_dir) / f"{idx:04}.jpg"
        img_file.rename(target_name)

<br/><br/><br/><br/>

## 2.設定

以下、入力方式によって適切なものを選んでください。

- 画像
  - [単一の画像ファイルのURLを指定する場合](#scrollTo=79RZXnYuuXSm)
  - [単一の画像ファイルをアップロードする場合](#scrollTo=RUnc5ujkGIG-)
  - [複数の既にダウンロード済みの画像ファイルを対象にする場合](#scrollTo=elmieBAGH9Bc)
- PDF
  - [単一のPDFファイルのURLを指定する場合](#scrollTo=OBluVMw1KSSd)
  - [単一のPDFファイルをアップロードする場合](#scrollTo=XtMeH4r1Kgw0)
  - [単一の既にダウンロード済みのPDFファイルを対象にする場合](#scrollTo=pTxI1yCnLDos)
- IIIF
  - [IIIFマニフェストファイルのURLを指定する場合](#scrollTo=Jh0O2Da_0Snv)

<br/><br/><br/><br/>

## 画像

<br/>

### 単一の画像ファイルのURLを指定する場合

- URL: 画像ファイルのURL
- 出力フォルダ: 出力するフォルダへのパス

入力サンプル：「源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 実行

URL = "https://dl.ndl.go.jp/api/iiif/2585098/R0000003/full/full/0/default.jpg" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndlkotenocr_v2/output/image_url" #@param {type:"string"}

task = Task(出力フォルダ)
task.download_image(URL, f"{str(1).zfill(4)}.jpg")
task.run_common_pipeline()

In [ ]:
# @title すべてのセルを実行するときに停止させるための処理
assert False, "このセルで実行を停止しました。"

<br/><br/><br/>

### 単一の画像ファイルをアップロードする場合

- 出力フォルダ: 出力するフォルダへのパス

以下の設定の再生ボタンを押すと、ファイルのアップロードフォームが表示されます。

In [ ]:
#@title 実行

出力フォルダ = "/content/drive/MyDrive/ndlkotenocr_v2/output/image_local" #@param {type:"string"}

task = Task(出力フォルダ)

# google colab依存
from google.colab import files
def upload_get_input_file():
  uploaded = files.upload()
  input_file = next(iter(uploaded))
  return input_file

input_file = upload_get_input_file()

# 画像の移動
opath = task.img_dir_path + "/" + f"{str(1).zfill(4)}.jpg"
!mv "{input_file}" "{opath}"

task.run_common_pipeline()

<br/><br/><br/>

### 複数の既にダウンロード済みの画像ファイルを対象にする場合

- 入力フォルダ: 入力するフォルダのパス
  - 指定したフォルダの下にimgフォルダを用意し、その中に画像を格納してください。
- 出力フォルダ: 出力するフォルダへのパス

In [ ]:
#@title 実行

入力フォルダ = "/content/drive/MyDrive/ndlkotenocr_v2/input" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndlkotenocr_v2/output/image_single" #@param {type:"string"}

task = Task(出力フォルダ)

入力フォルダ = str(Path(入力フォルダ))

files = glob.glob(入力フォルダ + "/img/*.jpg")
files.sort()

for i in range(len(files)):
  # 画像のコピー
  input_file = files[i]
  output_file = f"{task.temporary_directory}/img/{str(i + 1).zfill(4)}.jpg"
  !cp "{input_file}" {output_file}

task.run_common_pipeline()

<br/><br/><br/><br/>

## PDF

<br/><br/><br/>

### 単一のPDFファイルのURLを指定する場合

- url: PDFファイルのURL
- 出力フォルダ: 出力するフォルダへのパス
- ruby: ルビのテキスト化を行うか否か

入力サンプル：「東洋学芸雑誌」（人間文化研究機構国立国語研究所所蔵）

In [ ]:
#@title 設定
url = "https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndlkotenocr_v2/output/pdf_url" #@param {type:"string"}

task = Task(出力フォルダ)

# pdfのパス
pdf_path = Path(f"/content/tmp/{task.task_id}.pdf")

# ダウンロード
os.makedirs(os.path.dirname(pdf_path), exist_ok=True)
!curl {url} -o {pdf_path} --insecure

task.prepare_pdf(pdf_path)

task.run_common_pipeline()

<br/><br/><br/>

### 単一のPDFファイルをアップロードする場合

- 出力フォルダ: 出力するフォルダへのパス

以下の設定の再生ボタンを押すと、ファイルのアップロードフォームが表示されます。

In [ ]:
#@title 設定

出力フォルダ = "/content/drive/MyDrive/ndlkotenocr_v2/output/pdf_local" #@param {type:"string"}

task = Task(出力フォルダ)

pdf_path = upload_get_input_file()

task.prepare_pdf(pdf_path)
task.run_common_pipeline()

<br/><br/><br/>

### 単一の既にダウンロード済みのPDFファイルを対象にする場合

- 入力ファイル: PDFファイルのパス
- 出力フォルダ: 出力するフォルダへのパス

In [ ]:
#@title　設定
入力ファイル = "/content/drive/MyDrive/ndlkotenseki_ocr2/input/pdfs/2M5-4.pdf" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndlkotenseki_ocr2/output/pdf_local" #@param {type:"string"}

task = Task(出力フォルダ)
task.prepare_pdf(入力ファイル)
task.run_common_pipeline()

<br/><br/><br/><br/>

## IIIF

<br/>

### IIIFマニフェストファイルのURLを指定する場合

- IIIFマニフェストファイルのURL: IIIFマニフェストファイルのURL
- 出力フォルダ: 出力するフォルダへのパス
- 開始コマ数: 処理を開始するコマ。デフォルトは1。
- 終了コマ数: 処理を終了するコマ。デフォルトは5。-1にするとすべて。
- 画像ダウンロードの間隔_秒数: 画像ダウンロードの間隔（秒数）

入力サンプル：「源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 設定
import time
from tqdm import tqdm

IIIFマニフェストファイルのURL = "https://dl.ndl.go.jp/api/iiif/2585098/manifest.json"#@param {type:"string"}

出力フォルダ = "/content/drive/MyDrive/ndlkotenocr2/output/iiif" #@param {type:"string"}

開始コマ数 =   1 #@param {type:"number"}
終了コマ数 =   11 #@param {type:"number"}
画像ダウンロードの間隔_秒数 =   1 #@param {type:"number"}

task = Task(出力フォルダ)

df = requests.get(IIIFマニフェストファイルのURL).json()
context = df["@context"]

targets = []
services = []

if context == "http://iiif.io/api/presentation/2/context.json":
  canvases = df["sequences"][0]["canvases"]

  for i in range(len(canvases)):
    canvas = canvases[i]

    index = i + 1

    if not (index >= 開始コマ数 and (終了コマ数 == -1 or index <= 終了コマ数)):
      continue

    url = canvas["images"][0]["resource"]["@id"]

    targets.append({
        "url": url,
        "filename": f"{str(i + 1).zfill(4)}.jpg"
    })

    services.append(canvas["images"][0]["resource"]["service"])

else:
  pass

for target in tqdm(targets):
  task.download_image(target["url"], target["filename"])
  time.sleep(画像ダウンロードの間隔_秒数)

task.run_common_pipeline(services=services)

<br/><br/><br/>

# その他

<br/>

### zipファイルとtask_idを指定する

- 入力ファイル: imgフォルダを含む圧縮したzipファイルへのパス
- task_id: （通常はuuidとなるが、独自に指定する）
- 出力フォルダ: 出力するフォルダへのパス

In [ ]:
#@title 実行

入力ファイル = "/content/drive/MyDrive/genji/100421276.zip" #@param {type:"string"}
task_id = "100421276"  #@param {type:"string"}
出力フォルダ = f"/content/drive/MyDrive/genji/output/100421276" #@param {type:"string"}

manifest = "https://kokusho.nijl.ac.jp/biblio/100421276/manifest" #@param {type:"string"}

#######

!rm -rf {出力フォルダ}

task = Task(出力フォルダ, task_id = task_id)

tmp_dir = f"/content/tmp"

copied_path = f"{tmp_dir}/{task_id}/{task_id}.zip"

os.makedirs(os.path.dirname(copied_path), exist_ok=True)

!cp -rp {入力ファイル} {copied_path}

output_dir = f"{tmp_dir}/{task_id}"
!unzip {copied_path} -d {output_dir}

入力フォルダ = str(Path(output_dir))

files = glob.glob(入力フォルダ + "/img/*.jpg")
files.sort()

for i in range(len(files)):
  # 画像のコピー
  input_file = files[i]
  output_file = f"{task.temporary_directory}/img/{str(i + 1).zfill(4)}.jpg"
  !cp {input_file} {output_file}

task.run_common_pipeline()

# 画像を出力フォルダにコピー
opath = f"{出力フォルダ}/{task_id}/files"
os.makedirs(opath, exist_ok=True)
!cp -rp {入力フォルダ} {opath}

# URIの修正

manifest_o_path = f"{出力フォルダ}/{task_id}/iiif/manifest_a.json"
manifest_e_path =  f"{出力フォルダ}/{task_id}/iiif/manifest_ea.json"

manifest_org = requests.get(manifest).json()
with open(manifest_o_path, "r") as f:
  manifest_new = json.load(f)

canvases_org = manifest_org["sequences"][0]["canvases"]
canvases_new = manifest_new["items"]

for i in range(len(canvases_new)):
  # canvases_new[i]["images"][0]["resource"] = canvases_org[i]["images"][0]["resource"]
  resource = canvases_org[i]["images"][0]["resource"]
  canvases_new[i]["items"][0]["items"][0]["body"] = {
      "id": resource["@id"],
      "type": "Image",
      "format": "image/jpeg",
      "width": resource["width"],
      "height": resource["height"],
      "service": resource["service"]
  }

with open(manifest_e_path, "w") as f:
    json.dump(manifest_new, f)

root_path =task.docs_dir
output_manifest_alto_path = f"{root_path}/iiif/{task_id}/manifest_ea.json"

os.makedirs(os.path.dirname(output_manifest_alto_path), exist_ok=True)

!cp {manifest_e_path} {output_manifest_alto_path}

Task.show_mirador(output_manifest_alto_path)